In [1]:
import numpy as np

from multivariate_bernoulli import MultivariateBernoulli

In [2]:
%load_ext autoreload
%autoreload 2

# Parameterizing a multivariate Bernoulli
Consider an M-dimensional multivariate Bernoulli. For illustrative example, we'll use M=3. We can think of this as a univariate categorical random variable $X_{cat}$ with $2^M$ states (in our 3-dim example: 8 states). The event $V_{abc} = \{X_1=a, X_2=b, X_3=c\}$ in the multivariate Bernoulli is just a binary encoding $abc$ of the event that the categorical variable the $k$th state, where $k = 
\text{decode}(abc)$. For example $V_{011} = V^{cat}_3$.

As we'll see, there is benefit to structuring as a multivariate Bernoulli for the sake of extracting marginal and conditional probabilities of events though. For example, the event that $\{X_1=1, X_3=0\}$ can be denoted as something like $V_{1?0}$. Thus to enable us to compute the probability of something like $V_{1?0}$, we might not want to parameterize the probabilities of each of the $2^M$ states of the categorical random variable in a vector $P(X_{cat} = k) = \mu^{cat}_k$ where $\mu^{cat} \in [0, 1]^{2^M}$ and $\sum\limits_i \mu^{cat}_i = 1$, but instead use a rank-$M$ tensor so that $P(X_{cat} = k) = P(X = \text{encode}(k)) = \mu_{\text{encode}(k)}$ (for example, $P(X_{cat} = 5) = P(X = [1, 0, 1]) = \mu_{101}$. This gives us the tensor $\mu \in [0, 1]^{2 \times 2 \times \dots \times 2}$ where $\sum\limits_{i_1} \sum\limits_{i_2} \dots \sum\limits_{i_M} \mu_{i_1 i_2 \dots i_M} = 1$. From this construction, $\mu_{abc} = \mu^{cat}_{\text{decode}(abc)}$, i.e. $\mu_{010} = \mu^{cat}_{2}$.

## Marginal probabilities
Armed with our tensor $\mu$, we can easily marginalize the multivariate Bernoulli distribution. Take for example our earlier event $V_{1?0}$. If we want the probability of $V_{1?0}$, we have to marginalize over the second dimension that corresponds to the unknown $X_2$. The equation for this is $P(V_{1?0}) = \left(\sum\limits_j \mu_{ijk}\right)_{10}$. What we are doing here is marginalizing over the second dimension corresponding to $X_2$ to create a rank $3-1=2$ tensor that represents the marginal distribution over $X_1$ and $X_3$, and then we simply look up the event $X_1 = 1, X_3 = 0$ by indexing as above.

## Conditional probabilities
Since $P(X_1 = a, X_2=b \dots | X_3 = c, X_4 = d \dots) = \frac{P(X_1 = a, X_2 = b, \dots, X_3 = c, X_4 = d, \dots)}{P(X_3 = c, X_4 = d, \dots)}$, we can use the marginalization above to compute the numerator and denominator, and then we can get the conditional probability.

## Conditional distribution
Alternatively, we can create a new multivariate Bernoulli of lower dimension by conditioning on information. To do this, we can just slice our tensor by the conditions, compute the marginal probability of this event in the same way as above (which is equivalent to summing over our sliced sub-tensor), and divide our sliced sub-tensor by this probability (which normalizes all probabilities so that they sum to one once more).

## Creating $\mu$
For a multivariate Gaussian, there is no higher-order dependence than second-degree, so a mean vector and covariance matrix suffice. This means for dimensionality $M$, a multivariate Gaussian has a vector of size $M$ and a covariance matrix of size $M^2$ and that's it. It is fairly standard to estimate these parameters via log-likelihood maximization from a dataset, or to hand-design these parameters with a lot of zeros in the covariance matrix. For the multivariate Bernoulli, there is no condition barring higher than second-order interactions. This is why we parameterize to $O(2^M)$ parameters rather than $O(M^2)$. This is in contrast to the Ising model [Ising model](https://en.wikipedia.org/wiki/Ising_model) which [this paper on the multivariate Bernoulli distribution](https://arxiv.org/abs/1206.1874) depicts as more analagous to the multivariate Gaussian for binary variables, where there is an assumption of no higher-than-second-order interactions.

The above paper presents a log-likelihood based estimation procedure from data, but the question remains how best to intuitively parameterize the multivariate Bernoulli outcome probability tensor in the same way that we intuitively construct the mean vector and covariance matrix for a multivariate Gaussian. In my mind, it would be ideal to give a few conditional probabilities that describe the dependencies between dimensions of the multivariate Bernoulli, and then to use independence assumptions to fill in the probability tensor. I'll have to spend some more time thinking about which parameters and assumptions are needed to uniquely define an outcome probability tensor intuitively, though.

# Gender and weightlifting ability example
Let's look at how we can use this parameterization of a multivariate Bernoulli. Consider the case of where we have two features: $X_0 :=$ "gender is male" and $X_1 :=$ "good weightlifting ability". These features are Bernoulli random variables.

We shall consider the case where there is some statistical dependence between $X_0$ and $X_1$. To parameterize this, we can fix:
- the conditional mean $\mu_{lift | male} = E[X_1 | X_0 = 1]$
- a different conditional mean $\mu_{lift | female} = E[X_1 | X_0 = 0]$
- and the gender mean $\mu_{male} = E[X_0]$

Knowing these terms, we can calculate (by the law of total probability and Bayes theorem):
- the lifter mean $\mu_{lift} = E[X_0] = (1 - \mu_{male}) \mu_{lift | female} + \mu_{male} \mu_{lift | male}$
- the conditional mean $\mu_{male | lift} = E[X_0 | X_1 = 1] = \frac{ \mu_{lift | male} \mu_{male} }{\mu_{lift}}$
- the conditional mean $\mu_{male | nolift} = E[X_0 | X_1 = 0] = \frac{ (1 - \mu_{lift | male}) \mu_{male} }{1 - \mu_{lift}}$

We can also construct an outcome probability tensor:
- $\mu_{11} = \mu_{lift | male} \mu_{male}$
- $\mu_{01} = \mu_{lift | female} (1 - \mu_{male})$
- etc.

In [3]:
# define the parameters
mu_lift_female = 0.2
mu_lift_male = 0.8
mu_male = 0.5

# calculate the other known properties
mu_lift = (1 - mu_male) * mu_lift_female + mu_male * mu_lift_male
mu_male_lift = (mu_lift_male * mu_male) / mu_lift
mu_male_nolift = ((1 - mu_lift_male) * mu_male) / (1 - mu_lift)
print(f'mu_lift        = {mu_lift:.4f}')
print(f'mu_male_lift   = {mu_male_lift:.4f}')
print(f'mu_male_nolift = {mu_male_nolift:.4f}')

# compute the probability tensor
mu_lift_and_male = mu_lift_male * mu_male
mu_lift_and_female = mu_lift_female * (1 - mu_male)
mu_nolift_and_male = (1 - mu_lift_male) * mu_male
mu_nolift_and_female = (1 - mu_lift_female) * (1 - mu_male)
mu_matrix = np.array([[mu_nolift_and_female, mu_lift_and_female], [mu_nolift_and_male, mu_lift_and_male]])
X = MultivariateBernoulli(mu_matrix, ['is_male', 'is_good_lifter'])

print()

print('Probability tensor:')
print(X.mu)

print()

print('Probability table:')
display(X.to_table())

print()
print()

print('Correlation matrix:')
display(X.corr())

mu_lift        = 0.5000
mu_male_lift   = 0.8000
mu_male_nolift = 0.2000

Probability tensor:
[[0.4 0.1]
 [0.1 0.4]]

Probability table:


,is_male,is_good_lifter,p(X)
0,0,0,0.4
1,0,1,0.1
2,1,0,0.1
3,1,1,0.4




Correlation matrix:


,is_male,is_good_lifter
is_male,1.0,0.6
is_good_lifter,0.6,1.0


### Playing with the variable

In [4]:
X.prob({'is_male': True})

0.5

In [5]:
X.conditional({'is_male': True}).prob({'is_good_lifter': True})

0.8

In [6]:
X.conditional({'is_male': False}).prob({'is_good_lifter': True})

0.2

In [7]:
# new mu
mu = np.array([[0.1, 0.0], [0.4, 0.5]])
X = MultivariateBernoulli(mu, variable_names=['is_male', 'is_good_lifter'])
X.to_table()

,is_male,is_good_lifter,p(X)
0,0,0,0.1
1,0,1,0.0
2,1,0,0.4
3,1,1,0.5


In [8]:
X.corr()

,is_male,is_good_lifter
is_male,1.000000,0.333333
is_good_lifter,0.333333,1.000000


In [9]:
X_uncorrelated = X.joint_marginal()

print('Uncorrelated X probability:')
display(X_uncorrelated.to_table())

print()
print()

print('Uncorrelated X correlation:')
display(X_uncorrelated.corr())

Uncorrelated X probability:


,is_male,is_good_lifter,p(X)
0,0,0,0.05
1,0,1,0.05
2,1,0,0.45
3,1,1,0.45




Uncorrelated X correlation:


,is_male,is_good_lifter
is_male,1.0,0.0
is_good_lifter,0.0,1.0
